# **Imports &#8595;**

In [1]:
#!pip install einops --no-index --find-links=file:///kaggle/input/packages/einops/ 
#!pip install transformers --no-index --find-links=file:///kaggle/input/packages/transformers/ 
#!pip install sentence-transformers --no-index --find-links=file:///kaggle/input/packages/sentence-transformers/ 

!pip install einops
!pip install transformers==4.42.4
!pip install -U sentence-transformers
!jupyter nbextension disable --py widgetsnbextension


[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter c

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import heapq

In [3]:
import numpy as np
import pandas as pd 
import os
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import recall_score, f1_score, precision_score

# **Load Dataset &#8595;**

In [4]:
DATA_PATH = "datasets/eedi-mining-misconceptions-in-mathematics"
EXTERNAL_DATA_PATH = "datasets/eedi-external-dataset"

train_df = pd.read_csv(f'{EXTERNAL_DATA_PATH}/all_train.csv', index_col='QuestionId') #this contains the original dataset + an external dataset generated by a LLM
test_df = pd.read_csv(f'{DATA_PATH}/test.csv')
misconceptions_df = pd.read_csv(f'{DATA_PATH}/misconception_mapping.csv')

pd.options.display.max_colwidth = 300
display(train_df.head(20))
pd.options.display.max_colwidth = 50

,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId,source,MisconceptionAName,MisconceptionBName,MisconceptionCName,MisconceptionDName,OriginalQuestionId
QuestionId,,,,,,,,,,,,,,,,,,,,
0,856.0,Use the order of operations to carry out calculations involving powers,33.0,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0,original,NaN,NaN,NaN,"Confuses the order of operations, believes addition comes before multiplication",NaN
1,1612.0,Simplify an algebraic fraction by factorising the numerator,1077.0,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{m^{2}+2 m-3}{m-3} \)",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN,original,"Does not know that to factorise a quadratic expression, to find two numbers that add to give the coefficient of the x term, and multiply to give the non variable term\n","Thinks that when you cancel identical terms from the numerator and denominator, they just disappear","Does not know that to factorise a quadratic expression, to find two numbers that add to give the coefficient of the x term, and multiply to give the non variable term\n",NaN,NaN
2,2774.0,Calculate the range from a list of data,339.0,Range and Interquartile Range from a List of Data,B,"Tom and Katie are discussing the \( 5 \) plants with these heights:\n\( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \)\nTom says if all the plants were cut in half, the range wouldn't change.\nKatie says if all the plants grew by \( 3 \mathrm{~cm} \) each,...",Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0,original,Believes if you changed all values by the same proportion the range would not change,NaN,Believes if you changed all values by the same proportion the range would not change,Believes if you add the same value to all numbers in the dataset the range will change,NaN
3,2377.0,Recall and use the intersecting diagonals properties of a rectangle,88.0,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with different length sides can never be... ![A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.](),acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0,original,Does not know the properties of a rectangle,Does not know the properties of a rectangle,NaN,Does not know the properties of a rectangle,NaN
4,3387.0,Substitute positive integer values into formulae involving powers or roots,67.0,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find values in the table below. What is the value covered by the star? \begin{tabular}{|c|c|c|c|c|}\n\hline\( r \) & \( 1 \) & \( 2 \) & \( 3 \) & \( 4 \) \\\n\hline\( f \) & \( 6 \) & \( 15 \) & \( \color{gold}\bigstar \) & \\\n\hline\n\end{tabular},\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0,original,NaN,NaN,NaN,Thinks you can find missing values in a given table by treating the row as linear and adding on the difference between the first two values given.,NaN
5,2052.0,Identify a unit of area,75.0,Area of Simple Shapes,D,"James has answered a question on the area of a trapezium and got an answer of \( 54 \).\n\nBehind the star he has written the units that he used.\n\n\(\n54 \, \bigstar \n\)\n\nWhich of the following units could be correct?",\( m \),\( \mathrm{cm} \),\( \mathrm{km}^{3} \),\( \mathrm{mm}^{2} \),686.0,686.0,686.0,NaN,original,Does not know units of area should be squared,Does not know units of area should be squared,Does not know units of area should be squared,NaN,NaN
6,376.0,Conver

# **Data Preprocessing &#8595;**

In [5]:
def clean(example, columns):
    """
    Cleans the example from the Dataset
    Args:
        example: an example from the Dataset
        columns: columns that will be cleaned

    Returns: update example containing 'clean' columns

    """
    for col in columns:
        text = example[f'{col}']

        # Empty text
        if type(text) not in (str, np.str_) or text=='':
            example[f'clean_{col}'] = ''
            return example

        # 'text' from the example can be of type numpy.str_, let's convert it to a python str
        text = str(text).lower()

        # Clean the text
        text = re.sub("\"", " ", text) # removes the " from certain texts
        text = re.sub("\n", " ", text) # removes the multiple "\n" 
        text = re.sub(r"(\\\w+)(\W)", r" \1 \2", text) # matches with the LaTeX commands like "\hline{}",... and transforms them to " \hline {}"
        text = re.sub(r"([\(|\{|\[|\|])", r" \1", text) # matches every opening parenthesis types and puts spaces on their left
        text = re.sub(r"([\)|\}|\]])", r"\1 ", text) # matches every closing parenthesis types and puts spaces on their right
        text = re.sub(r"\\(?![a-zA-Z])", " ", text) # removes every backslash that is not the start of a LaTeX command
        text = re.sub(r"\( | \)", "", text) # removes the parentheses that appear sometimes from nowhere 
        text = re.sub(r"\[ | \]", "", text) # removes the parentheses that appear sometimes from nowhere
        
        text = re.sub(r" +", " ", text) # cleans the double spaces made by above substitutions
        # Update the example with the cleaned text
        example[f'clean_{col}'] = text.strip()
    return example

# testing_data = {
#     'QuestionText': ["This is a question with a newline\nin the middle"],
#     'AnswerAText': ["Answer A\nwith newline and \\table[test]"],
#     'AnswerBText': ["Answer B\nwith newline and \hline(uwo)"],
#     'AnswerCText': ["Answer C\nwith newline and \color{gold}"],
#     'AnswerDText': ["Answer D\nwith newline and \\begin{tabular}"]
# }
# df = pd.DataFrame(testing_data)
# df = df.apply(clean, axis = 1, columns = columns_to_clean)
# display(df.head(1))

columns_to_clean = ['QuestionText', 'AnswerAText', 'AnswerBText', 'AnswerCText', 'AnswerDText']
train_df = train_df.apply(clean, axis = 1, columns = columns_to_clean)

# Adjust column order
new_order = ['ConstructId', 'ConstructName', 'SubjectId', 'SubjectName', 'CorrectAnswer']
for col in columns_to_clean:
    new_order.append(col)
    new_order.append(f'clean_{col}')


misconception_related_columns = [
    'MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId',
    'MisconceptionAName', 'MisconceptionBName', 'MisconceptionCName', 'MisconceptionDName',
    'source', 'OriginalQuestionId'
]

misconception_data_df = train_df[misconception_related_columns]

train_df = train_df[new_order]


display_train_df = train_df[['QuestionText', 'clean_QuestionText','AnswerAText', 'clean_AnswerAText', 'AnswerBText', 'clean_AnswerBText', 'AnswerCText', 'clean_AnswerCText', 'AnswerDText', 'clean_AnswerDText']]
pd.options.display.max_colwidth = 300
display(display_train_df.head(1))
pd.options.display.max_colwidth = 50

,QuestionText,clean_QuestionText,AnswerAText,clean_AnswerAText,AnswerBText,clean_AnswerBText,AnswerCText,clean_AnswerCText,AnswerDText,clean_AnswerDText
QuestionId,,,,,,,,,,
0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,3 \times 2+4-5 where do the brackets need to go to make the answer equal 13 ?,\( 3 \times(2+4)-5 \),3 \times (2+4) -5,\( 3 \times 2+(4-5) \),3 \times 2+ (4-5),\( 3 \times(2+4-5) \),3 \times (2+4-5),Does not need brackets,does not need brackets


# **Pre-trained LLM Classifier &#8595;**

In [6]:
task_name_to_instruct = {"example": "Given a math question and a distractor incorrect answer, please retrieve the most accurate reason for the misconception.",}
query_prefix = "Instruct: "+task_name_to_instruct["example"]+"\nQuery: "
passage_prefix = ""

In [7]:
def create_query(example):
    """
    Creates a query from the features of the example
    Args:
        example: an example from the Dataset

    Returns: update example containing 'queryA/B/C/D' columns

    """
    # ConstructName, SubjectName, CorrectAnswer, QuestionText, AnswerText, source
    for answer in ['A', 'B', 'C', 'D']:
        if example['CorrectAnswer'] == answer:
            continue
        query = f"##Construct Name##: {example['ConstructName']}, ##Subject Name##: {example['SubjectName']}, ##Question Text##: {example['clean_QuestionText']}, ##Answer Text##: {example[f'clean_Answer{answer}Text']}"
        example[f'Query{answer}'] = query
    return example

#testing_data = {
#    'ConstructName': ["Use the order of operations to carry out calculations involving powers"],
#    'SubjectName': ["BIDMAS"],
#    'CorrectAnswer': ["A"],
#    'clean_QuestionText': ["3 \times 2+4-5 where do the brackets need to go to make the answer equal 13 ?"],
#    'clean_AnswerAText': ["3 \times (2+4) -5"],
#    'clean_AnswerBText': ["3 \times 2+ (4-5)"],
#    'clean_AnswerCText': ["3 \times (2+4-5)"],
#    'clean_AnswerDText': ["does not need brackets"],
#    'source': ["original"]
#}
#df = pd.DataFrame(testing_data)
#df = df.apply(create_query, axis = 1)

#pd.options.display.max_colwidth = 300
#display(df.head(1))
#pd.options.display.max_colwidth = 50
train_df = train_df.apply(create_query, axis = 1)
display(train_df[['QueryA', 'QueryB', 'QueryC', 'QueryD']].head(1))


,QueryA,QueryB,QueryC,QueryD
QuestionId,,,,
0,NaN,##Construct Name##: Use the order of operation...,##Construct Name##: Use the order of operation...,##Construct Name##: Use the order of operation...


In [8]:
#model = SentenceTransformer('/kaggle/input/Bert-MLM_arXiv-MP-class_zbMath')
model = SentenceTransformer('math-similarity/Bert-MLM_arXiv-MP-class_zbMath')


def encode_misconceptions(example): 
    if pd.isna(example['MisconceptionName']):
        example['Embeddings'] = np.nan
    else:
        example['Embeddings'] = model.encode([example['MisconceptionName']], show_progress_bar=False)
    return example

misconceptions_df = misconceptions_df.apply(encode_misconceptions, axis=1)
embeddings = misconceptions_df['Embeddings'].values.tolist()
display(misconceptions_df.head(5))

,MisconceptionId,MisconceptionName,Embeddings
0,0,Does not know that angles in a triangle sum to...,"[[-0.19901921, -0.3886316, 0.12010808, 0.65365..."
1,1,Uses dividing fractions method for multiplying...,"[[0.6487998, 0.98909736, -0.4768678, -0.153043..."
2,2,Believes there are 100 degrees in a full turn,"[[0.097909, 0.12341561, -0.37839338, 0.1686860..."
3,3,Thinks a quadratic without a non variable term...,"[[-0.039934896, 0.9032715, -0.29499432, 0.3320..."
4,4,Believes addition of terms and powers of terms...,"[[0.14368054, 0.9618122, -0.27214074, 0.149991..."


In [9]:
def encode_query(example):
    for column in ['QueryA', 'QueryB', 'QueryC', 'QueryD']: 
        if pd.isna(example[column]):
            example[f'Embed{column}'] = np.nan
        else:
            example[f'Embed{column}'] = model.encode([example[column]], show_progress_bar=False)
    return example

train_df = train_df.apply(encode_query, axis = 1)
display(train_df[['EmbedQueryA', 'EmbedQueryB', 'EmbedQueryC', 'EmbedQueryD']].head(1))

,EmbedQueryA,EmbedQueryB,EmbedQueryC,EmbedQueryD
QuestionId,,,,
0,NaN,"[[0.5036406, 0.85721004, -0.42123085, 0.390189...","[[0.5198178, 0.7997615, -0.4544851, 0.3832625,...","[[0.5535355, 0.8618221, -0.4446567, 0.27094868..."


In [10]:
def create_predictions(example):
    for answer in ['A', 'B', 'C', 'D']:
        if answer == example['CorrectAnswer']:
            example[f'top25_{answer}'] = np.nan
        else:                               
            top_25 = []
            for i, misconception in enumerate(misconceptions_df['Embeddings']):
                misconception = misconception.reshape(1, -1)
                query_embedding = example[f'EmbedQuery{answer}'].flatten()
                similarity = misconception @ query_embedding
                misconception_id = misconceptions_df.iloc[i]['MisconceptionId']
                if len(top_25) < 25:
                    #print(misconception.shape)
                    #print(example[f'EmbedQuery{answer}'].shape)
                    heapq.heappush(top_25, (similarity.item(), misconception_id))
                else:
                    heapq.heappushpop(top_25, (similarity.item(), misconception_id))
            top_25_misconceptions = [misconception_id for _, misconception_id in sorted(top_25, reverse=True)]
            example[f'top25_{answer}'] = top_25_misconceptions
    return example

train_df = train_df.apply(create_predictions, axis = 1)
display(train_df[['top25_A', 'top25_B', 'top25_C', 'top25_D']].head(5))

,top25_A,top25_B,top25_C,top25_D
QuestionId,,,,
0,NaN,"[1963, 1593, 791, 1825, 2098, 466, 1651, 59, 1...","[1963, 1593, 791, 1825, 77, 59, 1389, 466, 154...","[1963, 77, 1825, 1593, 791, 2270, 1026, 1571, ..."
1,"[363, 2398, 2484, 59, 1593, 80, 1540, 606, 182...","[363, 2398, 2484, 59, 1593, 80, 1540, 606, 182...","[363, 2398, 59, 2484, 1593, 80, 1540, 606, 182...",NaN
2,"[2205, 1328, 461, 1016, 709, 1744, 2257, 675, ...",NaN,"[2205, 1328, 461, 1016, 709, 1744, 675, 2455, ...","[2205, 1328, 461, 1016, 709, 1744, 675, 2455, ..."
3,"[955, 22, 382, 1016, 1298, 1947, 1553, 1170, 2...","[955, 22, 382, 1553, 1170, 1298, 1317, 496, 18...",NaN,"[22, 955, 382, 1553, 1170, 1845, 496, 1317, 12..."
4,NaN,"[847, 2242, 1341, 1270, 2368, 2308, 760, 1622,...","[847, 1341, 2242, 1270, 2368, 2308, 1622, 760,...","[847, 1341, 2242, 1270, 2368, 2308, 1622, 760,..."


In [ ]:
def map_at_k(y_true, y_pred, k=25):
    """
    Computes Mean Average Precision at K (MAP@K).
    
    Args:
        y_true: List of lists containing true misconception IDs for each observation.
        y_pred: List of lists containing predicted misconception IDs for each observation.
        k: The maximum number of predictions to consider (default is 25).

    Returns:
        MAP@K score.
    """
    total_score = 0.0
    for true_labels, pred_labels in zip(y_true, y_pred):
        if not true_labels:  # skip if there are no true labels
            continue
        
        used_labels = set()  # track already scored labels
        pred_labels = pred_labels[:k]  # Consider only top K predictions
        score = 0.0
        hits = 0
        
        for i, pred in enumerate(pred_labels, start=1):
            if pred in true_labels and pred not in used_labels:  # skip already scored labels
                hits += 1
                used_labels.add(pred)
                score += hits / i  
        
        total_score += score / min(len(true_labels), k) 
    
    return total_score / len(y_true) if y_true else 0.0



def mean_reciprocal_rank(y_true, y_pred):
    """
    Computes Mean Reciprocal Rank (MRR).
    
    Args:
        y_true: List of lists containing true misconception IDs for each observation.
        y_pred: List of lists containing predicted misconception IDs for each observation.

    Returns:
        MRR score.
    """
    total_rr = 0.0
    for true_labels, pred_labels in zip(y_true, y_pred):
        if not true_labels or not pred_labels:  
            continue
        
        for rank, pred in enumerate(pred_labels, start=1):
            if pred in true_labels:  #
                total_rr += 1 / rank
                break
    
    return total_rr / len(y_true) if y_true else 0.0



def precision_at_k(y_true, y_pred, k=25):
    """
    Computes Precision at K (P@K).
    
    Args:
        y_true: List of lists containing true misconception IDs for each observation.
        y_pred: List of lists containing predicted misconception IDs for each observation.
        k: The maximum number of predictions to consider (default is 25).

    Returns:
        Precision at K score.
    """
    total_precision = 0.0
    for true_labels, pred_labels in zip(y_true, y_pred):
        if not true_labels:  # Skip if there are no true labels
            continue
        
        pred_labels = pred_labels[:k] 
        hits = sum(1 for pred in pred_labels if pred in true_labels)  
        total_precision += hits / k 
    
    return total_precision / len(y_true) if y_true else 0.0

def evaluate_model(y_true, y_pred, k=25):
    """
    Evaluates the model using MAP@K, MRR, and Precision@K.
    
    Args:
        y_true: List of lists containing true misconception IDs for each observation.
        y_pred: List of lists containing predicted misconception IDs for each observation.
        k: The maximum number of predictions to consider (default is 25).

    Returns:
        A dictionary containing the scores for MAP@K, MRR, and Precision@K.
    """
    metrics = {
        "MAP@K": map_at_k(y_true, y_pred, k),
        "MRR": mean_reciprocal_rank(y_true, y_pred),
        "Precision@K": precision_at_k(y_true, y_pred, k),
    }
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    return metrics


# Filter for rows where source is "original"
filtered_df = train_df.iloc[:1869]
misconception_data_filtered = misconception_data_df.iloc[:1869]


def extract_y_true(df):
    y_true = []
    for _, row in df.iterrows():
        true_misconceptions = []
        for answer in ['A', 'B', 'C', 'D']:
            if pd.notna(row[f'Misconception{answer}Id']):
                true_misconceptions.append(int(row[f'Misconception{answer}Id']))
        y_true.append(true_misconceptions)
    return y_true

y_true = extract_y_true(misconception_data_filtered)

def extract_y_pred(df):
    """
    Extracts the predicted misconceptions from the top25 columns.

    Args:
        df: DataFrame containing the predictions.

    Returns:
        y_pred: List of lists containing the predicted misconception IDs.
    """
    y_pred = []
    for _, row in df.iterrows():
        predictions = []
        for answer in ['A', 'B', 'C', 'D']:
            # Retrieve the value
            value = row[f'top25_{answer}']
            
            # Check if the value is a NumPy array or a list and not empty
            if isinstance(value, (np.ndarray, list)):
                predictions.extend(value if isinstance(value, list) else value.tolist())
        
        y_pred.append(predictions)
    return y_pred

 


y_pred = extract_y_pred(filtered_df)

# Evaluate predictions
metrics = evaluate_model(y_true, y_pred, k=25)

print("\nEvaluation Metrics  :")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

MAP@K: 0.0182
MRR: 0.0337
Precision@K: 0.0067

Evaluation Metrics for Rows 0-1868 (source: 'original'):
MAP@K: 0.0182
MRR: 0.0337
Precision@K: 0.0067


: 

# **Output &#8595;** 

In [ ]:
train_df = train_df.reset_index()
display(train_df.columns)
display(train_df.head(3))

Index(['QuestionId', 'AnswerAText', 'AnswerBText', 'AnswerCText',
       'AnswerDText', 'ConstructId', 'ConstructName', 'CorrectAnswer',
       'QueryA', 'QueryB', 'QueryC', 'QueryD', 'QuestionText', 'SubjectId',
       'SubjectName', 'clean_AnswerAText', 'clean_AnswerBText',
       'clean_AnswerCText', 'clean_AnswerDText', 'clean_QuestionText',
       'EmbedQueryA', 'EmbedQueryB', 'EmbedQueryC', 'EmbedQueryD', 'top25_A',
       'top25_B', 'top25_C', 'top25_D'],
      dtype='object')

,QuestionId,AnswerAText,AnswerBText,AnswerCText,AnswerDText,ConstructId,ConstructName,CorrectAnswer,QueryA,QueryB,...,clean_AnswerDText,clean_QuestionText,EmbedQueryA,EmbedQueryB,EmbedQueryC,EmbedQueryD,top25_A,top25_B,top25_C,top25_D
0,0,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,856.0,Use the order of operations to carry out calcu...,A,NaN,##Construct Name##: Use the order of operation...,...,does not need brackets,3 \times 2+4-5 where do the brackets need to g...,NaN,"[[0.50364035, 0.8572104, -0.42123145, 0.39019,...","[[0.519818, 0.79976207, -0.45448592, 0.3832621...","[[0.55353516, 0.86182225, -0.44465786, 0.27094...",NaN,"[1963, 1593, 791, 1825, 2098, 466, 1651, 59, 1...","[1963, 1593, 791, 1825, 77, 59, 1389, 466, 154...","[1963, 77, 1825, 1593, 791, 2270, 1026, 1571, ..."
1,1,\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,1612.0,Simplify an algebraic fraction by factorising ...,D,##Construct Name##: Simplify an algebraic frac...,##Construct Name##: Simplify an algebraic frac...,...,does not simplify,"simplify the following, if possible: \frac {m^...","[[0.7318566, 0.965311, -0.67278385, 0.3614066,...","[[0.74010336, 0.96261096, -0.67143726, 0.36470...","[[0.7320576, 0.96087164, -0.67774916, 0.370653...",NaN,"[363, 2398, 2484, 59, 1593, 80, 1540, 606, 182...","[363, 2398, 2484, 59, 1593, 80, 1540, 606, 182...","[363, 2398, 59, 2484, 1593, 80, 1540, 606, 182...",NaN
2,2,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,2774.0,Calculate the range from a list of data,B,##Construct Name##: Calculate the range from a...,NaN,...,neither is correct,tom and katie are discussing the 5 plants with...,"[[-0.23089972, -0.27076513, -0.46715936, 0.192...",NaN,"[[-0.21322936, -0.2692874, -0.4728039, 0.18241...","[[-0.22471593, -0.28712168, -0.45850548, 0.193...","[2205, 1328, 461, 1016, 709, 1744, 2257, 675, ...",NaN,"[2205, 1328, 461, 1016, 709, 1744, 675, 2455, ...","[2205, 1328, 461, 1016, 709, 1744, 675, 2455, ..."


In [ ]:
def create_output(example):
    for answer in ['A', 'B', 'C', 'D']:
        if example[f'top25_{answer}'] is np.nan:
            continue
        else: 
            misconceptions_str = " ".join(map(str, example[f'top25_{answer}'])) if isinstance(example[f'top25_{answer}'], list) else example[f'top25_{answer}']
            output_df.loc[len(output_df)] = [f"{example['QuestionId']}_{answer}", misconceptions_str]

output_df = pd.DataFrame(columns=['QuestionId', 'MisconceptionId'])
train_df.apply(create_output, axis = 1)
display(output_df.head(10))

,QuestionId,MisconceptionId
0,0_B,1963 1593 791 1825 2098 466 1651 59 1540 567 7...
1,0_C,1963 1593 791 1825 77 59 1389 466 1540 567 980...
2,0_D,1963 77 1825 1593 791 2270 1026 1571 606 2398 ...
3,1_A,363 2398 2484 59 1593 80 1540 606 1825 1963 88...
4,1_B,363 2398 2484 59 1593 80 1540 606 1825 1963 88...
5,1_C,363 2398 59 2484 1593 80 1540 606 1825 1963 88...
6,2_A,2205 1328 461 1016 709 1744 2257 675 1372 2455...
7,2_C,2205 1328 461 1016 709 1744 675 2455 1372 1684...
8,2_D,2205 1328 461 1016 709 1744 675 2455 1684 1372...
9,3_A,955 22 382 1016 1298 1947 1553 1170 2565 668 1...


In [ ]:
output_df.to_csv('submission.csv', index=False, header=['QuestionId_Answer', 'MisconceptionId']) 

In [ ]:
output_df

,QuestionId,MisconceptionId
0,0_B,1963 1593 791 1825 2098 466 1651 59 1540 567 7...
1,0_C,1963 1593 791 1825 77 59 1389 466 1540 567 980...
2,0_D,1963 77 1825 1593 791 2270 1026 1571 606 2398 ...
3,1_A,363 2398 2484 59 1593 80 1540 606 1825 1963 88...
4,1_B,363 2398 2484 59 1593 80 1540 606 1825 1963 88...
...,...,...
11210,eedi_1833_B,77 1963 2270 2285 1026 931 126 575 656 940 184...
11211,eedi_1833_C,77 1963 2270 2285 1026 931 126 575 656 940 462...
11212,eedi_674_A,1760 1170 1468 1298 1632 1372 358 663 1592 141...
11213,eedi_674_B,1760 1170 1468 1298 1632 358 1372 663 1410 190...
